In [ ]:
import matplotlib.pyplot as plt
from morse_helpers import convert_image_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, AveragePooling2D, MaxPooling1D, ZeroPadding2D
import tensorflow as tf
import numpy as np

        

In [ ]:
# Retrieve saved model
loaded_model = tf.keras.models.load_model("saved_model")

In [ ]:
def show_image(img):
    fig = plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()
    return fig

In [ ]:
# Add noise to image and predict and display
orig_img = convert_image_to_array("wordsMatrices_13_000/0.png", (5, 1400))

x = tf.reshape(orig_img, [1, 5, 1400, 1])
x = tf.keras.layers.GaussianNoise(0.2)(x)

noise_img_exp_dim = np.expand_dims(x[0], axis=0)
prediction = loaded_model.predict(noise_img_exp_dim)

pos_pred = int(prediction[0][0][0] * 1400)

cat_pred = np.argmax(prediction[1])

image_noise = x[0].numpy()

show_image(orig_img)
show_image(image_noise)

print("position", pos_pred)
print("letter", cat_pred)
